## Autograd
Autograd is a core component of PyTorch that provides automatic differentiation for tensor operations. It enables gradient computation, which is essential for  training machine learning models using optimization algorithms like gradient descent.

## Calculating Gradients mannualy (by making custom functions)

In [11]:
def dy_dx(x):
  return 2*x

In [12]:
dy_dx(3)

6

In [13]:
import math

def dz_dx(x):
    return 2 * x * math.cos(x**2)

In [14]:
dz_dx(4)

-7.661275842587077

## Calculating Gradients with Autograd

In [15]:
import torch

In [16]:
x = torch.tensor(3.0, requires_grad=True)
x

tensor(3., requires_grad=True)

In [17]:
y = x**2            # makes a computaion graph (x -> sqr -> y)
y

tensor(9., grad_fn=<PowBackward0>)

In [18]:
y.backward()      

In [19]:
x.grad

tensor(6.)

In [20]:
x = torch.tensor(4.0, requires_grad=True)
x

tensor(4., requires_grad=True)

In [21]:
y = x ** 2
y

tensor(16., grad_fn=<PowBackward0>)

In [22]:
z = torch.sin(y)         # computation graph (x -> sqr -> y -> sin -> z)
z                        # x - leaf node, y - intermediate node, z - root node

tensor(-0.2879, grad_fn=<SinBackward0>)

In [23]:
z.backward()

In [24]:
x.grad

tensor(-7.6613)

In [25]:
y.grad               # gradients cannot be calculated for intermediate nodes by default

/tmp/ipykernel_55/2678329692.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  y.grad               # gradients cannot be calculated for intermediate nodes by default


## Basic NN with mannual gradients

In [26]:
import torch

# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [27]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [28]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [29]:
loss

tensor(6.7012)

In [30]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [31]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


## Basic NN with Autograd

In [32]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

In [33]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [34]:
w

tensor(1., requires_grad=True)

In [35]:
b

tensor(0., requires_grad=True)

In [36]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [37]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [38]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [39]:
# Compputation Graph

#              b
#              |
# w -- (*) -- (+) -- z - (sigmoid) -- y -- (loss func) -- Loss
#       |
#       x

In [40]:
loss.backward()

In [41]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


## Autograd on nd Tensor

In [42]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

In [43]:
x

tensor([1., 2., 3.], requires_grad=True)

In [44]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [45]:
y.backward()

In [46]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

## Clearing gradients 

In [47]:
x = torch.tensor(2.0, requires_grad=True)
for i in range(5):
    y = x ** 2
    y.backward()
    print(x.grad)       # By default gradients accumulate on running the backward pass again and again

tensor(4.)
tensor(8.)
tensor(12.)
tensor(16.)
tensor(20.)


In [48]:
x = torch.tensor(2.0, requires_grad=True)
for i in range(5):
    y = x ** 2
    y.backward()
    print(x.grad)
    x.grad.zero_()     # clearing gradient

tensor(4.)
tensor(4.)
tensor(4.)
tensor(4.)
tensor(4.)


## Disable Gradient Tracking

In [49]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [50]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [51]:
y.backward()

In [52]:
x.grad

tensor(4.)

In [53]:
# option 1 - requires_grad_(False)
x.requires_grad_(False)

tensor(2.)

In [54]:
y = x ** 2
y

tensor(4.)

In [55]:
# y.backward()    # will give error bc no gradient tracking

In [56]:
# option 2 - detach()
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [57]:
z = x.detach()
z

tensor(2.)

In [58]:
y = x ** 2

In [59]:
y

tensor(4., grad_fn=<PowBackward0>)

In [60]:
y1 = z ** 2
y1

tensor(4.)

In [61]:
y.backward()

In [62]:
# y1.backward()   # will give error bc no gradient tracking 

In [63]:
# option 3 - torch.no_grad()
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [64]:
with torch.no_grad():
    y = x ** 2

In [65]:
y

tensor(4.)

In [66]:
# y.backward()     # will give error bc no gradient tracking